In [6]:
from utilities import *
# 

In [15]:
# df = load_data_yf(symbol="EURUSD=X", start="2025-06-20", end="2025-08-17")
symbols = {
    'EURUSD': 'EURUSD=X',
    'USDCHF': 'USDCHF=X',
    'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X',
    'BTCUSD': 'BTC-USD',
    'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X',
    'XAGUSD': 'XAGUSD=X',
    'SP500m': '^GSPC',
    'UK100': '^FTSE'
}
ticker = symbols['EURUSD']
df = yf.download(ticker, start="2018-01-01", end="2025-10-15", interval="1d")

/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_28661/466460947.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2018-01-01", end="2025-10-15", interval="1d")
[*********************100%***********************]  1 of 1 completed


In [16]:
df = add_pivots(df, window=3)

In [17]:
df

Price,Close,High,Low,Open,Volume,pivoth,pivotl
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,,
Date,,,,,,,
2018-01-01,1.200495,1.201504,1.199904,1.200495,0,False,False
2018-01-02,1.201158,1.208094,1.200855,1.201086,0,False,False
2018-01-03,1.206345,1.206709,1.200495,1.206200,0,False,False
2018-01-04,1.201043,1.209190,1.200495,1.201129,0,True,False
2018-01-05,1.206884,1.208459,1.202154,1.206622,0,False,False
...,...,...,...,...,...,...,...
2025-10-08,1.165460,1.165297,1.160362,1.165460,0,False,False
2025-10-09,1.163196,1.164958,1.154921,1.163196,0,False,False


In [ ]:
plot_candles_with_pivots(df, 0, 200)

In [ ]:
LOOKBACK = 40 # 40
PIVOT_WINDOW = 3 # 3
MAX_HIGH_SLOPE = 0.0002 # 0.0001
MIN_LOW_SLOPE = 0.0005 # 0.0005
MIN_R2 = 0.7 # 0.7

In [ ]:
signal_len = 4        # breakout bar + 2 more

df["triangle_flag"] = False #np.nan

for i in range(30, len(df)):
    flag = detect_flag(df, i, 
                       lookback= LOOKBACK, 
                       pivot_window= PIVOT_WINDOW, 
                       max_high_slope= MAX_HIGH_SLOPE, 
                       min_low_slope = MIN_LOW_SLOPE,
                       min_r2= MIN_R2)
    if flag:
        dt = df.index[i]
        print(f"Flag confirmed with breakout bar {i}, {dt}")
        print(flag)
        end_pos = min(i + signal_len, len(df))
        df.iloc[i:end_pos, df.columns.get_loc("triangle_flag")] = True

In [ ]:
df

In [ ]:
mask = df["triangle_flag"]
label_index = df.index[mask]
int_index   = np.flatnonzero(mask.values)

In [ ]:
int_index

In [ ]:
from backtesting import Backtest, Strategy

df['my_signal'] = df['triangle_flag']

class ShortSignalStrategy(Strategy):
    """
    Short when df['my_signal'] is True.
    Stop-loss  = High + sl_buffer * High
    Take-profit = entry – rr * (stop−entry)
    """
    sl_buffer = 0.01     # 1 % above high
    rr        = 3      # risk-reward multiple

    def init(self):
        # cache series for speed
        self.signal = self.data.my_signal
        self.high   = self.data.High
        self.close  = self.data.Close

    def next(self):
        i = len(self.data) - 1

        # open short
        if self.signal[i] and not self.position:
            entry   = self.close[i]
            stop    = self.high[i] * (1 + self.sl_buffer)
            tp      = entry - self.rr * (stop - entry)
            self.sell(size=0.99, sl=stop, tp=tp)

bt = Backtest(
    df,
    ShortSignalStrategy,
    cash=10000,
    commission=0.0005,
    trade_on_close=True, # enter at close of signal bar
    exclusive_orders=True,
    margin=1/10
)

stats = bt.run()
stats

In [ ]:
# optimise the RR multiple if you wish
stats_opt = bt.optimize(rr=list(np.arange(1.5, 3.1, 0.5)))
stats_opt['_strategy']

In [ ]:
# Precompute the flags (stores lists of (index, price) in the breakout bar row)
df = precompute_triangle_flags(
    df,
    lookback=LOOKBACK, pivot_window=PIVOT_WINDOW,
    max_high_slope=MAX_HIGH_SLOPE, min_low_slope=MIN_LOW_SLOPE, min_r2=MIN_R2,
    signal_len=3,
)

In [ ]:
# Plot a region
plot_candles_with_pivots_and_flags(df, start_idx=3800, end_idx=4200)